앞서 배운 앞서 배운 피드 포워드 신경망은 입력의 길이가 고정되어 있어 자연어 처리를 위한 신경망으로는 한계가 있었음  
$\Longrightarrow$ 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요해졌음

# 1. 순환 신경망(Recurrent Neural Network, RNN)
> 입력과 출력을 시퀀스 단위로 처리하는 모델

## 1) 순환 신경망(Recurrent Neural Network, RNN)
- 앞서 배운 Feed Forward NN처럼 출력층 방향으로만 결과를 보내는 것 이외에도 다른 신경망이 있음
- 그 중 하나가 RNN.
- RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있음

<img src=rnn_image1_ver2.png width=200>

- $x$ : 입력층의 입력벡터, $y$ : 출력층의 출력벡터
- 메모리 셀(cell) : RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
    - 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행함
- 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 함
- 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받음
- 은닉 상태(hidden state) : 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값

<img src=rnn_image2_ver3.png width=300>

- 일반적으로 위의 그림에서 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하거나, 화살표 대신 여러 시점으로 펼쳐서 표현하기도 함

<img src=rnn_image3_ver2.png width=300>

- 입력과 출력의 길이를 위와 같이 다르게 설계할 수 있음

- Examples
    - 하나의 입력에 대해서 여러개의 출력(1-to-many) 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용될 수 있음
    - 단어 시퀀스를 입력하고 하나의 출력(many-to-1) 모델은 입력문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 스펨메일 판별(spam detection)에 사용 될 수 있음
    
<img src=rnn_image3.5.png width=200>

- Examples
    - 입력 문장으로부터 대답 문장을 출력하는 챗봇과 번역기, ch12에서 배우는 개체명 인식이나 품사 태깅과 같은 작업
  

<img src=rnn_image3.7.png width=200>

### RNN 수식

<img src=rnn_image4_ver2.png width=100>

- $h_t$ : 시점 t에서의 은닉 상태값
- 은닉층 : $h_t = tanh(W_xx_t + W_hh_{t-1} + b)$
- 출력층 : $y_t = f(W_yh_t + b)$, where $f$ : 비선형함수

---

<img src=rnn_images4-5.png width=500>

In NLP,
- $D_h$ : 은닉 상태의 크기, $d$ : 단어 벡터의 차원
- $x_t$ : 단어 벡터
- 활성화함수로 tanh가 주로 사용되지만 ReLU로 바꿔 사용하는 시도도 있음
- 출력층은 결과값인 $y_t$를 계산하기 위한 활성화함수를 선택할 때,
    - 이진 분류를 해야하는 경우 : 시그모이드함수
    - 다중 분류를 해야하는 경우 : 소프트맥스함수

## 2) 케라스(Keras)로 RNN 구현하기

In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN

### RNN 층을 추가하는 코드.
model.add(SimpleRNN(hidden_size))

### 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))
- hidden_size = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.
- timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.
- input_dim = 입력의 크기.


### 다른 표기
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N)) (**단, M과 N은 정수**)

<img src=rnn_image6between7.png width=400>

- RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받음
    - batch_size는 한 번에 학습하는 데이터의 개수
- 위의 코드는 출력층까지 포함한 하나의 완성된 인공 신경망 코드가 아니라 은닉층. 즉, RNN 층에 대한 코드

RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냅니다.
- 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다.
- 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다.
- 이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정이 가능합니다. (output_dim은 앞서 코드에서 정의한 hidden_size의 값으로 설정됩니다.)

<img src=rnn_image8_ver2.png width=300>

- time step=3일 때, return_sequences = True를 설정했을 때와 그렇지 않았을 때 어떤 차이가 있는지를 보여줌
    - return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력하며, 
    - 별도 기재하지 않거나 return_sequences=False로 선택할 경우에는 메모리 셀은 하나의 은닉 상태값만을 출력합니다.
    

마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 은닉층이 하나 더 있는 경우이거나 many-to-many 문제를 풀 수 있습니다.

뒤에서 배우는 LSTM이나 GRU도 내부 메커니즘은 다르지만 model.add()를 통해서 층을 추가하는 코드는 사실상 SimpleRNN 코드와 같은 형태를 가집니다. 실습을 통해 모델 내부적으로 출력 결과를 어떻게 정의하는지 보면서 RNN을 이해해보겠습니다.

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN

Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


### 위에서 parameter 구하는 것이 이해가 잘 안되서 가져온 예제
- model.add(SimpleRNN(4, input_shape=(2,3)) 
<img src=71An9.png width=500>

(number of parameters) = (num of units)$^2$ + (num of units * input dim) + (num of units)  
= **(num of units) * (num of units + input dim + 1)**

따라서, 여기서는 답이 $4 * (4+3+1) = 32$

In [4]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


## 3) 파이썬으로 RNN 구현하기
> Numpy로 RNN층 구현할 예정

- 앞서 메모리 셀에서 은닉상태를 계산하는 식은 다음과 같았음
$h_t = tanh(W_xX_t+W_hh_{t-1}+b)$

RNN 층을 실제 동작되는 코드로 구현해보겠습니다. 아래의 코드는 이해를 돕기 위해 (timesteps, input_dim) 크기의 2D 텐서를 입력으로 받았다고 가정하였으나, 실제로 케라스에서는 (batch_size, timesteps, input_dim)의 크기의 3D 텐서를 입력으로 받는 것을 기억합시다.

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
# (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wx = np.random.random((hidden_size, input_dim))

# (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size))

# (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).
b = np.random.random((hidden_size,))

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


이제 모든 시점의 은닉 상태를 출력한다고 가정하고, RNN 층을 동작시켜봅시다.

In [4]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.90334389 0.94037776 0.93814014 0.94480089 0.59620645 0.81155734
  0.87225354 0.7660213 ]
 [0.9990606  0.99956121 0.9998413  0.9999273  0.99966701 0.99921067
  0.9999504  0.99956405]
 [0.99964944 0.99983138 0.99996936 0.9999911  0.99994521 0.9997723
  0.99999281 0.99990743]
 [0.99988896 0.9999175  0.99999389 0.99999744 0.99998548 0.99993063
  0.99999831 0.99995577]
 [0.99989531 0.99992301 0.99999089 0.99999686 0.99997626 0.9999323
  0.99999693 0.99996231]
 [0.99998449 0.99998146 0.99999912 0.99999947 0.99999629 0.99998498
  0.99999942 0.99999187]
 [0.99952903 0.99967433 0.99993981 0.9999844  0.99991312 0.99969677
  0.99998057 0.99985735]
 [0.99992321 0.99990797 0.99999551 0.99999804 0.99998886 0.99996038
  0.99999853 0.9999572 ]
 [0.99991038 0.99980649 0.99998826 0.99999602 0.99997536 0.99995488
  0.99999292 0.99993889]
 [0.99983748 0.99993604 0.99998673 0.99999517 0.99996986 0.99980697
  0.999995   0.99996505]]


## 4) BPTT(Backpropagation through time, BPTT)

해당 내용은 추가 예정입니다. 몰라도 이후 실습에 문제 없는 부분이므로 무시하고 학습하세요.

## 5) 깊은 순환 신경망(Deep Recurrent Neural Network)

<img src=rnn_image4.5_finalPNG.png width=300>

앞서 RNN도 다수의 은닉층을 가질 수 있다고 언급한 바 있습니다. 위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다. 은닉층을 2개 추가하는 경우 코드는 아래와 같습니다.

In [ ]:
model = Sequential()
model.add(SimpleRNN(hidden_size, return_sequences = True))
model.add(SimpleRNN(hidden_size, return_sequences = True))

위의 코드에서 첫번째 은닉층은 다음 은닉층이 존재하므로 **return_sequences = True**를 설정하여 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.

## 6) 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
>양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반

- Example
    - Exercise is very effective at [______] belly fat.
    
'운동은 복부 지방을 [ ] 효과적이다'라는 영어 문장이고, 정답은 reducing(줄이는 것)입니다. 그런데 위의 영어 빈 칸 채우기 문제를 잘 생각해보면 정답을 찾기 위해서는 이전에 나온 단어들만으로는 부족합니다. 목적어인 belly fat(복부 지방)를 모르는 상태라면 정답을 결정하기가 어렵습니다.

즉, RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측하지만 실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많습니다. 그래서 이전 시점의 데이터뿐만 아니라, 이후 시점의 데이터도 힌트로 활용하기 위해서 고안된 것이 양방향 RNN입니다.

<img src=rnn_image5_ver2.png width=400>

- **양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용**
- 첫번째 메모리 셀은 앞에서 배운 것처럼 **앞 시점의 은닉 상태(Forward States)**를 전달받아 현재의 은닉 상태를 계산(<font color=orange>주황색 메모리 셀</font>)
- 두번째 메모리 셀은 **뒤 시점의 은닉 상태(Backward States)**를 전달 받아 현재의 은닉 상태를 계산(<font color=green>초록색 메모리 셀</font>)
- 이 두 개의 값 모두가 출력층에서 출력값을 예측하기 위해 사용됌

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

물론, 양방향 RNN도 다수의 은닉층을 가질 수 있습니다. 아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.

<img src=rnn_image6_ver3.png width=400>

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))